In [18]:
#
# Inspired by https://www.kaggle.com/lpkirwin/fivethirtyeight-elo-ratings
#

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

df = pd.read_csv('./data/RegularSeasonCompactResults.csv')
year = 2017

In [19]:
df_sp = df.loc[df['Season'] == year]

In [20]:
team_ids = set(df_sp.WTeamID).union(set(df_sp.LTeamID))
len(team_ids)

351

In [21]:
from glicko2 import Player

In [22]:
def glicko_rounds(glicko, preds, df):
    for row in df_sp.itertuples():
        w = row.WTeamID
        l = row.LTeamID

        w_rating, l_rating = glicko[w].getRating(), glicko[l].getRating()
        w_rd, l_rd = glicko[w].getRd(), glicko[l].getRd()

        #
        # We are predicting if the team with the lower ID wins.
        # Glicko would predict the lower ID team to win if their glicko score was higher. 
        #
        if w_rating > l_rating:
            preds.append(1.)
        else:
            preds.append(0.)

        glicko[w].update_player([l_rating], [l_rd], [1])
        glicko[l].update_player([w_rating], [w_rd], [0])

In [23]:
glicko = dict(zip(list(team_ids), [Player() for _ in range(len(team_ids))]))
preds = []
glicko_rounds(glicko, preds, df_sp)
sum(preds) / len(preds)  # Prediction accuracy of Glicko2 in regular season. 

0.6646895273401298

In [24]:
preds = []
df = pd.read_csv('./data/ConferenceTourneyGames.csv')
df_sp = df.loc[df['Season'] == year]
glicko_rounds(glicko, preds, df_sp)
sum(preds) / len(preds)  # Accuracy of Glicko2 in Conference tourney, conditioned on regular season.

0.8129251700680272

In [25]:
preds = []
df = pd.read_csv('./data/NCAATourneyCompactResults.csv')
df_sp = df.loc[df['Season'] == year]
glicko_rounds(glicko, preds, df_sp)
sum(preds) / len(preds)  # Accuracy of Glicko2 in NCAA tourney, conditioned on regular season and conf. tourney. 

0.6417910447761194